In [ ]:
# Getting the cholera data

import pandas as pd
import os

In [ ]:
import pandas as pd
import os

#Load data from drive
cases_df = pd.read_parquet("/content/drive/MyDrive/CSCI 436 Data/data/CHOLERA_0000000001.parquet") #Change path to your data location
deaths_df = pd.read_parquet("/content/drive/MyDrive/CSCI 436 Data/data/CHOLERA_0000000002.parquet") #Change path to your data location
fatality_df = pd.read_parquet("/content/drive/MyDrive/CSCI 436 Data/data/CHOLERA_0000000003.parquet") #Change path to your data location

#Rename value columns to what they represent
cases_df = cases_df.rename(columns={"Value": "Reported Cholera Cases"})
deaths_df = deaths_df.rename(columns={"Value": "Reported Cholera Deaths"})
fatality_df = fatality_df.rename(columns={"Value": "Cholera Case Fatality Rate"})

#Select columns for merging
merge_keys = ["SpatialDim", "TimeDim"]
cases_df = cases_df[merge_keys + ["Reported Cholera Cases"]]
deaths_df = deaths_df[merge_keys + ["Reported Cholera Deaths"]]
fatality_df = fatality_df[merge_keys + ["Cholera Case Fatality Rate"]]

#Merge the df's into one
combined_df = cases_df.merge(deaths_df, on=merge_keys, how="outer")
combined_df = combined_df.merge(fatality_df, on=merge_keys, how="outer")
combined_df = combined_df.sort_values(by=merge_keys)
combined_df.rename(columns={'TimeDim': 'Year', 'SpatialDim': 'Country'}, inplace=True)

#Split by year and indicator
output_dir = "/content/drive/MyDrive/CSCI 436 Data/data/cholera_by_year"
os.makedirs(output_dir, exist_ok=True)

for year in combined_df["Year"].dropna().unique():
    year_df = combined_df[combined_df["Year"] == year]

    for indicator, col_name in {
        "cases": "Reported Cholera Cases",
        "deaths": "Reported Cholera Deaths",
        "fatality_rate": "Cholera Case Fatality Rate"
    }.items():
        output_path = os.path.join(output_dir, f"{indicator}_{int(year)}.csv")
        export_df = year_df[["Country", "Year", col_name]].dropna(subset=[col_name])
        export_df.to_csv(output_path, index=False)

All files created successfully.
